In [1]:
from glob import glob
import cPickle
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
data_dirs = glob("/home/ruoxu/workspace/data/public/cifar-10-batches-py/data*")

###Get train, test, label and reshape images

In [2]:
def reshape_img(im):
    tmp = im.reshape((3,32,32))
    im_reshape = np.zeros((32,32,3))
    for i in range(0,3):
        im_reshape[:,:,i] = tmp[i]
    return im_reshape

In [3]:
X =[]
labels_train = []
for d_dir in data_dirs:
    with open(d_dir) as f:
        tmp = cPickle.load(f)
        X.append(tmp['data'])
        labels_train.append(tmp['labels'])
X = np.vstack(X)
labels_train = np.concatenate(labels_train)
print X.shape
print labels_train.shape

X_train = [reshape_img(im) for im in X]
X_train = np.array(X_train)
print X_train.shape

(50000, 3072)
(50000,)
(50000, 32, 32, 3)


In [4]:
with open("/home/ruoxu/workspace/data/public/cifar-10-batches-py/test_batch") as f:
    tmp = cPickle.load(f)
    X = tmp['data']
    labels_test = tmp['labels']
labels_test = np.array(labels_test)
print X.shape
X_test = [reshape_img(im) for im in X]
X_test = np.array(X_test)
print X_test.shape
print labels_test.shape

(10000, 3072)
(10000, 32, 32, 3)
(10000,)


In [5]:
from sklearn.cross_validation import train_test_split
X_train,X_val,labels_train,labels_val = train_test_split(X_train, labels_train, test_size = 0.1)
print X_train.shape,X_val.shape
print labels_train.shape, labels_val.shape

(45000, 32, 32, 3) (5000, 32, 32, 3)
(45000,) (5000,)


###Extract image feature from Decaf

In [8]:
from IPython.parallel import Client
client = Client()
view = client.load_balanced_view()
view.block = True
len(client)

20

In [9]:
def feature_extractor(im):
    from decaf.scripts.imagenet import DecafNet
    f_dir = '/home/ruoxu/workspace/data/public/imagenet_pretrained/'
    net = DecafNet(f_dir + 'imagenet.decafnet.epoch90',f_dir + 'imagenet.decafnet.meta')
    scores = net.classify(im.astype('uint8'),center_only=True)
    feature = net.feature('fc6_cudanet_out')
    return feature.flatten()

In [ ]:
%time X_train_feats = view.map(feature_extractor,X_train)
X_train_feats = np.array(X_train_feats)
print X_train_feats.shape

In [ ]:
%time X_test_feats = view.map(feature_extractor,X_test)
X_test_feats = np.array(X_test_feats)
print X_test_feats.shape

#load data

In [15]:
import cPickle
with open("data/train_features") as f:
    %time X_train_feats = cPickle.load(f)
with open("data/train_labels") as f:
    %time labels_train = cPickle.load(f)
with open("data/test_features") as f:
    %time X_test_feats = cPickle.load(f)
with open("data/test_labels") as f:
    %time labels_test = cPickle.load(f)

CPU times: user 2min 34s, sys: 14.6 s, total: 2min 49s
Wall time: 2min 49s
CPU times: user 132 ms, sys: 18 µs, total: 132 ms
Wall time: 132 ms
CPU times: user 30.7 s, sys: 479 ms, total: 31.2 s
Wall time: 31.2 s
CPU times: user 18.7 ms, sys: 0 ns, total: 18.7 ms
Wall time: 18.7 ms


In [16]:
from collections import Counter
print Counter(labels_train)
print Counter(labels_test)

Counter({0: 5000, 1: 5000, 2: 5000, 3: 5000, 4: 5000, 5: 5000, 6: 5000, 7: 5000, 8: 5000, 9: 5000})
Counter({0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000})


In [17]:
with open("/home/ruoxu/workspace/data/public/cifar-10-batches-py/batches.meta") as f:
    label_names = cPickle.load(f)['label_names']

In [18]:
# dog out
import numpy as np
X = np.concatenate((X_train_feats,X_test_feats))
labels = np.concatenate((labels_train, labels_test))
print X.shape, labels.shape
ind_test = np.where(labels == 5)[0]
ind_train = np.where(labels != 5)[0]
X_train_feats = X[ind_train]
X_test_feats = X[ind_test]
labels_train = labels[ind_train]
labels_test = labels[ind_test]
print X_train_feats.shape, X_test_feats.shape
print labels_train.shape, labels_test.shape

(60000, 4096) (60000,)
(54000, 4096) (6000, 4096)
(54000,) (6000,)


In [19]:
from gensim.models import Word2Vec
%time model = Word2Vec.load_word2vec_format("/home/ruoxu/opt/word2vec/word2vec.bin", binary=True)
label_dict = {}
for i in range(len(label_names)):
    label_dict[i] = model[label_names[i]]

CPU times: user 3.22 s, sys: 83.7 ms, total: 3.3 s
Wall time: 3.3 s


In [22]:
tmp_train = [label_dict[i] for i in labels_train]
tmp_test = [label_dict[i] for i in labels_test]
labels_train = np.array(tmp_train)
labels_test = np.array(tmp_test)
print labels_train.shape, labels_test.shape

(54000, 100) (6000, 100)


In [23]:
from sklearn.cross_validation import train_test_split
X_train_feats,X_val_feats,labels_train,labels_val = train_test_split(X_train_feats, labels_train, test_size = 0.1)
print X_train_feats.shape,X_val_feats.shape
print labels_train.shape, labels_val.shape

(48600, 4096) (5400, 4096)
(48600, 100) (5400, 100)


In [25]:
##normalize
from sklearn.preprocessing import StandardScaler
ss_X = StandardScaler()
ss_Y = StandardScaler()
ss_X.fit(X_train_feats)
ss_Y.fit(labels_train)
X_train_feats, X_val_feats = ss_X.transform(X_train_feats), ss_X.transform(X_val_feats)
labels_train, labels_val = ss_Y.transform(labels_train), ss_Y.transform(labels_val)
print X_train_feats.shape
print labels_train.shape
print X_val_feats.shape
print labels_val.shape

(48600, 4096)
(48600, 100)
(5400, 4096)
(5400, 100)


In [27]:
import h5py
import os
save_dir = "/home/ruoxu/workspace/x/text-image-caffe/data"
train_file_name = os.path.join(save_dir, "train.h5")
test_file_name = os.path.join(save_dir,"test.h5")
val_file_name = os.path.join(save_dir,"val.h5")

import os
with h5py.File(train_file_name, 'w') as f:
    f['data'] = X_train_feats
    f['label'] = labels_train.astype(np.float32)
with h5py.File(test_file_name, 'w') as f:
    f['data'] = X_test_feats
    f['label'] = labels_test.astype(np.float32)
with h5py.File(val_file_name,"w") as f:
    f['data'] = X_val_feats
    f['label'] = labels_val.astype(np.float32)
with open(os.path.join(save_dir, "train.txt"), 'w') as f:
    f.write(train_file_name + '\n')
with open(os.path.join(save_dir,"test.txt"), 'w') as f:
    f.write(test_file_name + '\n')
with open(os.path.join(save_dir,"val.txt"), 'w') as f:
    f.write(val_file_name + '\n')
with open(os.path.join(save_dir,"x_normalizer"),"w") as f:
    cPickle.dump(ss_X,f)
with open(os.path.join(save_dir,"y_normalizer"),"w") as f:
    cPickle.dump(ss_Y,f)